In [ ]:
from network import get_sts_model
from triplet_dataset import TripletDataset
model_path = 'ammarnasr/LoRa_all-MiniLM-L12-v1'
model = get_sts_model(model_path)
tokenizer = model.tokenizer

In [ ]:
train_dataset = TripletDataset(None, tokenizer=tokenizer, using_allnli=True)
sample_batch = next(iter(train_dataset))


In [ ]:
import modeltraining 
from peft import LoraConfig
rank = 64
peft_config = LoraConfig(inference_mode=False,
            r=rank,
            lora_alpha=rank*2,
            lora_dropout=0.05,
            target_modules=['value','query','key', 'dense']
            )

training_args = {'use_allnli': True, 'peft_config': peft_config}
modeltraining.main(training_args)